# Data Combining & Cleaning Notebook

In [359]:
import pandas as pd
import numpy as np

##### Data Cleaning Explanation
It's necessary to separate out the long-stay and short-stay residents since they have separate measures. I'd expect they would also have very different outcomes in this data since they have different experiences within the Skilled Nursing Facilities. 

For the purposes of this analysis, I'll be focusing on long-stay patients.
 
##### Why use CSVs and not the CMS apis available? 
I would have liked to use the CMS apis to make the data ingestion process cleaner and more easily replicated by another user of this file. However, given the fact that I am using so much data, I decided it wouldn't be wise to use their apis to export data when there is a bulk download option in the form of csv files.

## Helper Functions

In [360]:
def count_individuals(grp):
    return np.sum(grp=='Individual')

def count_organization(grp):
    return np.sum(grp=='Organization')

def new_owner(grp):
    return np.sum(grp==True)
    
def duplicated_names(names):
    List_of_names = names.split(',')
    non_dups = set(List_of_names)
    if len(List_of_names) == len(non_dups):
        return False
    else:
        return True

## Importing Data Files

### Loading & Cleaning Quality Data

In [361]:
class import_and_clean:
    '''
    Ingests CMS data from CSVs containing 'NH_QualityMsr_MDS' in name and cleans data.
    
    Class inputs are data_location, data_year, and the resident type (short-term vs long term).
    '''
    def __init__(self, data_location, data_year, resident_type):
        self.data_location = data_location
        self.year = data_year
        self.resident_type = resident_type
        
    def read_file(self):
        df = pd.read_csv(self.data_location, encoding='cp1252', dtype={'Federal Provider Number': 'str'})

        #Separating out residents by group: Long vs Short-stay patients
        stay = df[df['Measure Description'].str.contains(self.resident_type)]

        #Remove currently unnecessary fields
        stay = stay[['Federal Provider Number', 'Provider Name', 'Measure Code','Measure Description',
                     'Q1 Measure Score','Q2 Measure Score','Q3 Measure Score','Q4 Measure Score','Four Quarter Average Score']]

        #Combine description & code into one field for ease of use
        stay['code_description'] = stay['Measure Code'].astype(str) + ' ' + stay['Measure Description']
        return stay
    
    def pivot_quarters(self):
        '''
        Pivot data from long to wide format
        '''
        df = self.read_file()
        #set filter for quarter based on input
        quarter_filter = 'Four Quarter Average Score'
        #pivot
        pivot_df = pd.pivot(df, columns = ['code_description'], 
                            index = ['Federal Provider Number','Provider Name'], 
                            values = [quarter_filter])
        #reset column names & add new columns
        pivot_df.columns = pivot_df.columns.get_level_values(1)
        pivot_df = pivot_df.reset_index()
        pivot_df['Quarter'] = quarter_filter.upper()
        pivot_df['Year'] = self.year
        return pivot_df

### Loading & Cleaning Provider Data

In [362]:
#load provider info
def load_provider_info(filepath):
    '''
    Intakes provider information csv.
    Outputs cleaned, shortened dataframe.
    '''
    df = pd.read_csv(filepath ,encoding='cp1252')
    df = df[['Federal Provider Number','Provider Address','Provider City', 'Provider State', 'Provider Zip Code',
       'Provider County Name', 'Ownership Type', 'Number of Certified Beds','Average Number of Residents per Day','Provider Type',
       'Provider Resides in Hospital', 'Date First Approved to Provide Medicare and Medicaid Services','Continuing Care Retirement Community', 'Special Focus Status',
       'Abuse Icon', 'Most Recent Health Inspection More Than 2 Years Ago','With a Resident and Family Council','Overall Rating','Health Inspection Rating',
       'Staffing Rating', 'RN Staffing Rating','Reported Nurse Aide Staffing Hours per Resident per Day',
       'Reported LPN Staffing Hours per Resident per Day','Reported RN Staffing Hours per Resident per Day',
       'Reported Licensed Staffing Hours per Resident per Day','Reported Total Nurse Staffing Hours per Resident per Day',
       'Reported Physical Therapist Staffing Hours per Resident Per Day','Case-Mix Nurse Aide Staffing Hours per Resident per Day',
       'Case-Mix LPN Staffing Hours per Resident per Day','Case-Mix RN Staffing Hours per Resident per Day',
       'Case-Mix Total Nurse Staffing Hours per Resident per Day','Adjusted Nurse Aide Staffing Hours per Resident per Day',
       'Adjusted LPN Staffing Hours per Resident per Day', 'Adjusted RN Staffing Hours per Resident per Day',
       'Adjusted Total Nurse Staffing Hours per Resident per Day','Number of Facility Reported Incidents','Number of Substantiated Complaints',
       'Number of Citations from Infection Control Inspections', 'Number of Fines', 'Total Amount of Fines in Dollars','Total Number of Penalties']]
    df['Federal Provider Number']= df['Federal Provider Number'].astype(str)
    df['Date First Approved to Provide Medicare and Medicaid Services'] = pd.to_datetime(df['Date First Approved to Provide Medicare and Medicaid Services'])
    df['time_diff']= (pd.Timestamp.now() - pd.to_datetime(df['Date First Approved to Provide Medicare and Medicaid Services'], 
                              errors='coerce')).dt.days
    
    return df

prov_df = load_provider_info('data/NH_ProviderInfo_Apr2021.csv')

In [363]:
# Facility Quality Data
import_file = import_and_clean('data/NH_QualityMsr_MDS_Apr2022.csv', '2021', 'long-stay')
df = import_file.pivot_quarters()

#Provider Data
prov_df = load_provider_info('data/NH_ProviderInfo_Apr2021.csv')

In [364]:
print(df.shape)
print(prov_df.shape)

(15203, 18)
(15328, 42)


In [365]:
# I decided to keep only facilities that had values in both data sets
# This removed 64 facilities
quality_provider = df.merge(prov_df, on = 'Federal Provider Number', how = 'inner')

In [366]:
print("Merge results in loss of ", quality_provider.shape[0] - df.shape[0], ' measures')

Merge results in loss of  -64  measures


### Loading & Cleaning Ownership Data

In [367]:
def ownership_dataset(data_path):
    '''
    Function to clean ownership dataset.
    
    Intakes csv. Outputs formatted dataframe.
    '''
    owner = pd.read_csv(data_path, encoding='cp1252')
    
    #create column that notes whether or not ownership information has been provided
    owner['ownership_info'] = np.where(owner['Role played by Owner or Manager in Facility']!='Ownership Data Not Available',
                                               'Available', 'Unavailable')

    #create column for date from when owner was first associated
    owner[['first_part', 'Associated_Since']] = owner['Association Date'].str.split('since ', expand = True)
    owner['Associated_Since'] = pd.to_datetime(owner['Associated_Since'])

    #create column indicating whether owner is new since start of 2021
    owner['Owner_new_2021'] = np.where(owner['Associated_Since'] >= pd.Timestamp(2021, 1, 1, 0), True, False)

    #create ownership length column
    owner['Ownership Length'] = pd.Timestamp.now().normalize() - owner['Associated_Since']
    owner['Ownership Length'] = owner['Ownership Length'].dt.days

    #drop unneeded columns
    owner = owner.drop(['first_part', 'Associated_Since', 'Association Date'], axis=1)

    #Get count of facilites per owner
    owner_count = owner.groupby('Owner Name').agg({'Federal Provider Number': lambda x: x.nunique()}).reset_index()
    owner_count = owner_count.rename({'Federal Provider Number': 'Count_Facilities_Owned'}, axis=1)
    owner_m = owner.merge(owner_count, on = 'Owner Name', how = 'left')
    
    #Group to owner level
    owner_group = owner_m.groupby(['Federal Provider Number', 
                     'Provider Name', 
                     'Owner Name']).agg({'Count_Facilities_Owned': 'max',
                                         'Owner_new_2021': 'max',
                                         'Ownership Length': 'max',
                                         'Owner Type': 'max',
                                         'Role played by Owner or Manager in Facility': lambda x: x.nunique()}).reset_index()
    #Pull out last name
    owner_group['Owner Last Name'] = owner_group['Owner Name'].str.split(',').str[0]
    
    #Group to facility level
    final_owner = owner_group.groupby(['Federal Provider Number', 
                         'Provider Name']).agg({'Owner Name': lambda x: x.nunique(),
                                                'Owner Last Name': lambda x: ','.join(x),
                                                'Owner Type': [count_individuals, count_organization],
                                                'Count_Facilities_Owned': ['max', 'mean'],
                                                'Owner_new_2021': new_owner,
                                                'Ownership Length': ['max', 'mean'],
                                                'Role played by Owner or Manager in Facility': ['max', 'mean']}).reset_index()
    
    #Rename columns
    final_owner.columns = final_owner.columns.get_level_values(0)
    final_owner.columns = ['Federal Provider Number', 'Provider Name', 'Number of Owners','Owner Names',
                          '# of Individual Owners', '# of Organization Owners',
                          'Max Facilities Owned', 'Mean Facilities Owned',
                          'New Owners Since 2021', 'Max Ownership Length', 'Mean Ownership Length',
                          'Max Roles', 'Mean Roles']
    
    #owners with same last name
    final_owner['Owners w Same Last Name'] = final_owner['Owner Names'].apply(duplicated_names)
    #facilities with both organizations and individuals listed on ownership
    final_owner['Individual & Org Owned'] = np.where((final_owner['# of Individual Owners']>0) & 
                                                     (final_owner['# of Organization Owners']>0), 
                                                     'Yes', 'No')
    final_owner = final_owner.drop('Owner Names', axis=1)
    return final_owner

In [370]:
#Ownership Information
owner = ownership_dataset('data/NH_Ownership_Apr2022.csv')

In [372]:
# I chose an inner merge here because the majority of the records that 
# were not in the ownership dataset had either all nulls or had a majority
# of nulls in the dataset

In [373]:
all_measures = quality_provider.merge(owner, on = ['Federal Provider Number', 'Provider Name'])

In [375]:
all_measures.to_csv('data/all_measures.csv')